In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

DAYS_IN_YEAR = 252
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


def get_ticker_from_yahoo(ticker: str, start: datetime, end: datetime):
    """ Get ticker from Yahoo Finance.
    """
    return yf.Ticker(ticker).history(start=start, end=end)


def get_ticker_data(ticker: str, start: datetime, end: datetime):
    """ Get the details stats for a stock ticker.
    
    Args:
        ticker (str): the ticker symbol on Yahoo Finance.
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        Dataframe: contains the daily stats of the ticker.
    """
    data = get_ticker_from_yahoo(ticker, start, end)
    data['Change'] = data['Close'].pct_change()
    data['Gross Return'] = data['Change'] + 1
    data['Log Return'] = np.log(data['Gross Return'])
    return data


def get_risk_free_return(start: datetime, end: datetime):
    """ Get risk-free return.

    The risk-free return is yielded by 13 Week Treasuray Bill.
    Note that the data from Yahoo Finance represent is annualized return.
    Each daily value contains annualized return %.

    Args:
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        Dataframe: contains the daily stats of risk-free return.
    """
    irx = get_ticker_from_yahoo('^IRX', start ,end)
    irx['Annaul Log Return'] = np.log(1 + (irx['Close'] / 100)) # divide by 100 since it's %
    irx['Log Return'] = irx['Annaul Log Return'] / DAYS_IN_YEAR
    return irx


def get_sharp_ratio(ticker: str, start: datetime, end: datetime):
    """ Get sharp ratio for a stock ticker.
    
    Args:
        ticker (str): the ticker symbol on Yahoo Finance.
        start (datetime): start time of data.
        end (datatime): end time of data.

    Returns:
        float: the sharp ratio of the ticker data.
    """
    ticker_data = get_ticker_data(ticker, start, end)
    irx_data = get_risk_free_return(start, end)
    ticker_data['Excess Return'] = ticker_data['Log Return'] - irx_data['Log Return']

    excess_return_mean = np.mean(ticker_data['Excess Return'])
    excess_return_std = np.std(ticker_data['Excess Return'])
    return excess_return_mean / excess_return_std



In [21]:

sharp_sp500 = get_sharp_ratio('^GSPC', start, end)

tickers = ['MSFT', 'AAPL', 'AMZN', 'NVDA', 'BETZ', 'VGT', 'IBUY', 'ARKK', 'ARKG', 'ARKW', 'ARKQ', 'ARKF', 'CLOU', 'ESPO', 'XBI', 'XLV', 'XPH', 'XLP', 'XLY', 'XRT', 'ITB', 'XLF', 'IYT', 'ITA', 'XOP', 'XLE', 'VNQ', 'XLB', 'TAN', 'XME', 'JETS']
start = datetime(2020, 4, 9)
end = datetime(2020, 9, 23)

datas = []
for ticker in tickers:
    data = {}
    data['name'] = ticker
    data['sharp'] = get_sharp_ratio(ticker, start, end)
    data['relative_sharp'] = data['sharp'] - sharp_sp500
    datas.append(data)

print(f'{"SP500":6} sharp: {sharp_sp500:.6f}')
datas.sort(key= lambda data: data['relative_sharp'], reverse=True)
for data in datas:
    print(f'{data["name"]:6} sharp: {data["sharp"]:.6f} || relative_sharp: {data["relative_sharp"]:.6f}')



SP500  sharp: 0.115277
IBUY   sharp: 0.312674 || relative_sharp: 0.197397
ARKF   sharp: 0.280109 || relative_sharp: 0.164832
ARKW   sharp: 0.255015 || relative_sharp: 0.139738
ESPO   sharp: 0.241484 || relative_sharp: 0.126207
TAN    sharp: 0.237422 || relative_sharp: 0.122145
ARKK   sharp: 0.235704 || relative_sharp: 0.120427
ARKG   sharp: 0.201813 || relative_sharp: 0.086536
XRT    sharp: 0.200940 || relative_sharp: 0.085663
ARKQ   sharp: 0.199834 || relative_sharp: 0.084557
NVDA   sharp: 0.195975 || relative_sharp: 0.080698
XLY    sharp: 0.193831 || relative_sharp: 0.078554
AAPL   sharp: 0.192539 || relative_sharp: 0.077262
CLOU   sharp: 0.180409 || relative_sharp: 0.065132
ITB    sharp: 0.173852 || relative_sharp: 0.058575
AMZN   sharp: 0.158544 || relative_sharp: 0.043267
VGT    sharp: 0.153796 || relative_sharp: 0.038519
IYT    sharp: 0.140279 || relative_sharp: 0.025002
BETZ   sharp: 0.135008 || relative_sharp: 0.019731
XBI    sharp: 0.126391 || relative_sharp: 0.011114
XLB    s

In [ ]:
data = pd.Series([1, 3, 8])
dataframe = pd.DataFrame({'A': [1,2,3,4], 'B': [3,3,3,3]})
dataframe2 = pd.DataFrame({'A': [1,2,3,4,5,6,7,8], 'B': [3,3,3,3,6,6,6,6]})
print (dataframe - dataframe2)

In [ ]:

df = pd.DataFrame(np.ones((6, 4)), index=pd.date_range('20130108', periods=6), columns=list('ABCD'))
df2 = pd.DataFrame(np.zeros((20, 4)), index=pd.date_range('20130101', periods=20), columns=list('ABCD'))

df2
print (df)
print (df2)
print (df['D'] - df2['D'])
print (np.std(df['D'] - df2['D']))
# print (np.mean(df - df2))